# Sequence to Sequence Networks in CNTK

## Background

Andrej Karpathy has a nice visualization of the five paradigms of neural network architectures:

<img src=http://karpathy.github.io/assets/rnn/diags.jpeg width=750px>

In this tutorial, we are going to be talking about the fourth paradigm: many-to-many, also known as sequence-to-sequence networks. The input is a sequence with a dynamic length, and the output is also a sequence with some dynamic length. It is the logical extension of the many-to-one paradigm in that previously we were predicting some category (which could easily be one of `V` words where `V` is an entire vocabulary) and now we want to predict a whole sequence of those categories.

The applications of sequence-to-sequence networks are nearly limitless. It is a natural fit for machine translation (e.g. English input sequences, French output sequences); automatic text summarization (e.g. full document input sequence, summary output sequence); word to pronunciation models (e.g. character [grapheme] input sequence, pronunciation [phoneme] output sequence); and even parse tree generation (e.g. regular text input, flat parse tree output).

## Basic theory

A sequence-to-sequence model consists of two main pieces: (1) an encoder; and (2) a decoder. Both the encoder and the decoder are recurrent neural network (RNN) layers that can be implemented using a vanilla RNN, an LSTM, or GRU cells. (here we will use LSTM). In the basic sequence-to-sequence model, the encoder processes the input sequence into a fixed representation that is fed into the decoder as a context. The decoder then uses some mechanism (discussed below) to decode the processed information into an output sequence. The decoder is a language model that is augmented with some "strong context" by the encoder, and so each symbol that it generates is fed back into the decoder for additional context (like a traditional LM). For an English to German translation task, the most basic setup might look something like this: 

<img src=pix/s2s.png width=700px>

The basic sequence-to-sequence network passes the information from the encoder to the decoder by initializing the decoder RNN with the final hidden state of the encoder as its initial hidden state. The input is then a "sequence start" tag (`<s>` in the diagram above) which primes the decoder to start generating an output sequence. Then, whatever word (or note or image, etc.) it generates at that step is fed in as the input for the next step. The decoder keeps generating outputs until it hits the special "end sequence" tag (`</s>` above).

A more complex and powerful version of the basic sequence-to-sequence network uses an attention model. While the above setup works well, it can start to break down when the input sequences get long. At each step, the hidden state `h` is getting updated with the most recent information, and therefore `h` might be getting "diluted" in information as it processes each token. Further, even with a relatively short sequence, the last token will always get the last say and therefore the thought vector will be somewhat biased/weighted towards that last word. To deal with this problem, we use an "attention" mechanism that allows the decoder to look not only at all of the hidden states from the input, but it also learns which hidden states, for each step in decoding, to put the most weight on. We will discuss an attention implementation at the end of this lab.

## Problem: Grapheme-to-Phoneme Conversion

The [grapheme](https://en.wikipedia.org/wiki/Grapheme) to [phoneme](https://en.wikipedia.org/wiki/Phoneme) problem is a translation task that takes the letters of a word as the input sequence (the graphemes are the smallest units of a writing system) and outputs the corresponding phonemes; that is, the units of sound that make up a language. In other words, the system aims to generate an unambigious representation of how to pronounce a given input word.

### Example

| Letters  | T | A | N | G | L | E |
| --- | --- |
| Phonemes | ~T | ~AE | ~NG | ~G | ~AHL | null |



## Problem: English-to-French Translation

The machine translation problem is well-known and easy to understand: given some English language sentence E (our input sequence), translate it to the equivalent French sentence F (our output sequence). MT researchers have been working on this problem for decades, coming up with ever-more complex systems to eke our the next fraction of a BLEU point. However, sequence-to-sequence networks have in some ways made a lot of their work antiquated (see, e.g. [Neural Machine Translation by Jointly Learning to Align and Translate](http://arxiv.org/abs/1409.0473)).

### Example (from Hansard):
| Language | Sentence | Source |
| --- | --- | --- |
| English | Fueling this growth in royalty revenues is the United States demand, which some day may place our Canadian domestic needs at risk. | hansard.36.2.house.debates.077.e |
| French | Les États-Unis aimeraient bien transformer cette croissance en redevances, ce qui pourrait mettre nos propres besoins en périls. | hansard.36.2.house.debates.077.f |


### Pre-processing and CNTKTextFormat

...

## Step 0: import all of the required functionality...

In [1]:
import numpy as np
import sys
import os
from cntk import Trainer, Axis, save_model, load_model #, text_format_minibatch_source, StreamConfiguration
from cntk.io import MinibatchSource, CTFDeserializer, StreamDef, StreamDefs, INFINITELY_REPEAT, FULL_DATA_SWEEP
from cntk.device import cpu, set_default_device
from cntk.learner import momentum_sgd, momentum_schedule
from cntk.ops import input_variable, cross_entropy_with_softmax, classification_error, sequence, slice, past_value, future_value, element_select, alias, hardmax
from cntk.ops.functions import CloneMethod
from cntk.graph import find_nodes_by_name

### Step 0.1: set our model hyperparameters...

In [5]:
cntk_dir = "c:\repos\cntk"                                          # data resides in the CNTK folder
data_dir = cntk_dir + "/Examples/SequenceToSequence/CMUDict/Data/"  # under Examples/SequenceToSequence
model_dir = "./Models"
input_vocab_size = 69
label_vocab_size = 69

# model dimensions
input_vocab_dim  = input_vocab_size
label_vocab_dim  = label_vocab_size
hidden_dim = 256
num_layers = 1

## Step 1: setup the input to the network

### Dynamic axes in CNTK

One of the important concepts in understanding CNTK is the idea of two types of axes: static axes, which are the traditional axes of a variable's shape, and dynamic axes, which have dimensions that are unknown until the variable is bound to real data at computation time. They are particularly important in the world of recurrent neural networks. Instead of having to decide a maximum sequence length ahead of time, padding your time to that size, and wasting computation, CNTK's dynamic axes allow for variable sequence lengths that are automatically padded in minibatches to be as efficient as possible.

When setting up sequences, there are two dynamic axes that are important to consider. The first is the batch axis, which is the axis along which multiple sequences are batched. The second is the dynamic axis particular to that sequence. The latter is specific to a particular input because of variable sequence lengths in your data. For example, in sequence to sequence networks, we have two sequences: the input sequence, and the ouptput (or 'label') sequence. One of the things that makes this type of network so powerful is that the length of the input sequence and the output sequence do not have to correspond to each other. Therefore, both the input sequence and the output sequence require their own unique dynamic axis.

When defining the input to a network, we set up the required dynamic axes and the shape of the input variables. Below, we define the shape (vocabulary size) of the inputs, create their dynamic axes, and finally create input variables that represent input nodes in our network.

In [6]:
# Source and target inputs to the model
batch_axis = Axis.default_batch_axis()
input_seq_axis = Axis('inputAxis')
label_seq_axis = Axis('labelAxis')

input_dynamic_axes = [batch_axis, input_seq_axis]
raw_input = input_variable(shape=(input_vocab_dim), dynamic_axes=input_dynamic_axes, name='raw_input')

label_dynamic_axes = [batch_axis, label_seq_axis]
raw_labels = input_variable(shape=(label_vocab_dim), dynamic_axes=label_dynamic_axes, name='raw_labels')

### Questions

1. Why do the shapes of the input variables correspond to the size of our dictionaries in sequence to sequence networks?

## Step 2: define the network

As discussed before, the sequence-to-sequence network is, at its most basic, an RNN encoder followed by an RNN decoder, and a dense output layer. We could do this in a few lines with the layers library, but let's go through things in a little more detail without adding too much complexity. The first step is to perform some manipulations on the input data; let's look at the code below and then discuss what we're doing.

In [7]:
# Instantiate the sequence to sequence translation model
input_sequence = raw_input

# Drop the sentence start token from the label, for decoder training
label_sequence = slice(raw_labels, label_seq_axis, 
                       1, 0, name='label_sequence') # <s> A B C </s> --> A B C </s>
label_sentence_start = sequence.first(raw_labels)   # <s>

is_first_label = sequence.is_first(label_sequence)  # 1 0 0 0 ...
label_sentence_start_scattered = sequence.scatter(
    label_sentence_start, is_first_label)

We have two input variables, `raw_input` and `raw_labels`. Typically, the labels would not have to be part of the network definition because they would only be used in a criterion node when we compare the network's output with the ground truth. However, in sequence-to-sequence networks, the labels themselves form part of the input to the network during training as they are fed as the input into the decoder.

To make use of these input variables, we will pass them through computation nodes. We first set `input_sequence` to `raw_input` as a convenience step. We then perform several modifications to `label_sequence` so that it will work with our network. For now you'll just have to trust that we will make good use of this stuff later.

First, we slice the first elements off of `label_sequence` so that it's missing the sentence-start token. This is because the decoder will always first be primed with that token, both during training and evaluation. When the ground truth isn't fed into the decoder, we will still feed in a sentence-start token, so we want to consistently view the input to the decoder as a sequence that starts with an actual value.

Then, we get `label_sequence_start` by getting the `first` element from the sequence `raw_labels`. This will be used to compose a sequence that is the first input to the decoder regardless of whether we're training or decoding. Finally, the last two statements set up an actual sequence, with the correct dynamic axis, to be fed into the decoder. The function `sequence.scatter` takes the contents of `label_sentence_start` (which is `<s>`) and turns it into a sequence with the first element containing the sequence start symbol and the rest of the elements containing 0's.

### Exercise 1: Create the encoder

We will use the LSTM implementation from `examples/common/nn.py`. Its function signature is:

`def LSTMP_component_with_self_stabilization(input, output_dim, cell_dim, recurrence_hookH=past_value, recurrence_hookC=past_value):`

and it returns a tuple `(hidden_state, hidden_cell)`.

1. Create the encoder (set the `output_dim` and `cell_dim` to `hidden_dim` which we defined earlier).
2. Set `num_layers` to something higher than 1 and create a stack of LSTMs to represent the encoder.
3. Get the output of the encoder and put it into the right form to be passed into the decoder [hard]

## Training

For sequence-to-sequence networks, the loss we use is cross-entropy. 

## Putting it all together



## Attention

An important extension to sequence-to-sequence models, especially when dealing with long sequences, is to use an attention mechanism. The idea behind attention is to allow the decoder, first, to look at any of the hidden state outputs from the encoder (instead of using only the final hidden state), and, second, to learn how much attention to pay to each of those hidden states given the context. This allows the outputted word at each time step `t` to depend not only on the final hidden state and the word that came before it, but instead on a weighted combination of *all* of the input hidden states!

...

The attention vector at output time $t$ over the input words $(1, ..., T_A)$ is defined as:

$$u_i^t = v^T tanh( W_1 h_i + W_2 d_t)$$
$$a_i^t = softmax(u_i^t)$$
$$d'_t = \sum_{i=1}^{T_A}{a_i^t h_i}$$

...

<img src=http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/12/Screen-Shot-2015-12-30-at-1.23.48-PM.png width=550px>

## Greedy decoding

Once we have a trained model, we want 


## Solutions

### Exercise 1

...


### Exercise 2

...